# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

### Questões

1. Quais os grupos possuem mais aderência as campanhas?
2. Qual o tipo de campanha que possui mais aderência?
3. Detalhe entre cada grupo, quais são as campanhas mais aderentes.
4. Qual a quantidade de ofertas que os usuários não viram?
5. Qual a quantidade de ofertas não lidas pelos usuários, porém validadas na compra?
6. Qual a quantidade de ofertas vistas pelos usuários, porém não validas na compra?

***To-do:***
* Crie ranges de idade para cada usuário

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [827]:
import pandas as pd
import numpy as np
import math
import json
#% matplotlib inline

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [845]:
portfolio.head(10)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [843]:
transcript[transcript['person'] == '78afa995795e4d85b5d9ceeca43f5fef']

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
15561,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6
47582,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},132
47583,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132
49502,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},144
53176,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168
85291,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},216
87134,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.67},222
92104,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 29.72},240
141566,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 23.93},378


In [830]:
def clean_portfolio(portfolio):
    '''
    Clean the portfolio dataset:
    - transform the "duration" column to hours
    - one-hot-enconde both channels and offer_type columns
    - rename the "id" column to "offer_id"
    
    INPUT:
    - the original portfolio dataset
    
    OUTPUT:
    - the cleaned portfolio dataset (portfolio_clean) 
    
    '''
    #create a copy
    portfolio_clean = portfolio.copy()
    
    #transform the "duration" column into hours
    portfolio_clean['duration'] = portfolio_clean['duration']*24
    
    #rename "id" column
    portfolio_clean = portfolio_clean.rename(columns={'id': 'offer_id'})

    #one-hot-encode the "channels" column
    portfolio_clean = portfolio_clean.drop('channels', axis = 1).join(portfolio_clean.channels.str.join('|').str.get_dummies())

    #one-hot-enconde the "offer_type" column
    offers_type = pd.get_dummies(portfolio_clean['offer_type'])

    #drop the old "offer_type" column and join dummies
    portfolio_clean = portfolio_clean.drop('offer_type', axis = 1).join(offers_type)
    
    return portfolio_clean
    

In [831]:
def clean_profile(profile):
    '''
    Clean the profile dataset:
    - change de dtype for the "became_member_on" to datetime
    - one-hot-enconde the "gender" column
    - rename the "id" column to "user_id"

    INPUT:
    - the original profile dataset
    
    OUTPUT:
    - the cleaned profile dataset (profile_clean) 
    
    '''
    #create a copy
    profile_clean = profile.copy()

    #change dtype on the "became_member_on" column to datetime
    profile_clean['became_member_on'] = pd.to_datetime(profile_clean['became_member_on'].astype(str), format='%Y%m%d')

    #one-hot-enconde the "gender" column
    genders = pd.get_dummies(profile_clean['gender'])
    
    #drop the "gender" column and join dummies
    profile_clean = profile_clean.drop('gender', axis = 1).join(genders)

    #rename the "id" column to "user_id"
    profile_clean = profile_clean.rename(columns={'id': 'user_id'})
    
    return profile_clean
    

In [900]:
def clean_transcript(transcript):
    '''
    Clean the transcript dataset:
    - normalize the "value" column
    - rename "person", "offer id" and "offer_id" columns
    - consolidate all offers ids into one column
    - drop the unnecessary columns 
    - drop the "reward" column --> it will be replaced after the final merge
    
    INPUT:
    - the original profile dataset
    
    OUTPUT:
    - the cleaned profile dataset
    
    '''
    #create a copy
    transcript_clean = transcript.copy()
    
    #normalizing dict row into columns
    transcript_clean = transcript_clean.join(pd.json_normalize(transcript_clean.value))  

    #rename columns
    transcript_clean = transcript_clean.rename(columns={'person': 'user_id',
                                                        'offer id': 'offer_id_received',
                                                        'offer_id': 'offer_id_completed'
                                                       }
                                              )
    #get the offer_id data from both columns
    transcript_clean['offer_id'] = np.where(transcript_clean['offer_id_received'].notna() == True,
                                            transcript_clean['offer_id_received'],
                                            transcript_clean['offer_id_completed']
                                           )
    
    #drop columns
    transcript_clean = transcript_clean.drop(columns=['value',
                                                      'reward',
                                                      'offer_id_received',
                                                      'offer_id_completed'],
                                             axis = 1
                                            )
    
    return transcript_clean


In [901]:
portfolio_df = clean_portfolio(portfolio)
transcript_df = clean_transcript(transcript)
profile_df = clean_profile(profile)

In [902]:
portfolio_df.head()

,reward,difficulty,duration,offer_id,email,mobile,social,web,bogo,discount,informational
0,10,10,168,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0,0
1,10,10,120,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0
2,0,0,96,3f207df678b143eea3cee63160fa8bed,1,1,0,1,0,0,1
3,5,5,168,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,1,0,0
4,5,20,240,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,0,1,0


In [904]:
transcript_df.sort_values('offer_id', ascending=True)

,user_id,event,time,amount,offer_id
10627,3c1e9922033e41189c6beb76c6e3edc0,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
114191,b390be062396490093121338df43f692,offer received,336,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
165659,504290d601394e6e8125490580594eaa,offer completed,408,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
114189,1aaf0bb7f4f34c828a75c62caf4140ce,offer received,336,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
157117,4ea6530ba3a54fb286b431f959a9527b,offer received,408,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,NaN
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,NaN
